# Opdracht 3.4: Locks

Er zijn verschillende manieren om race conditions te voorkomen. Eén daarvan is het gebruik van locks. In het voorbeeld hiervoor willen we feitelijk voorkomen dat het tweede proces (welke thread dat ook zou mogen zijn) de 'verkeerde' waarde van value uitleest, omdat het andere proces deze aan het aanpassen is. Het proces dat als eerste de waarde uitleest, zal een lock moeten leggen op value. (In andere talen wordt dit ook wel een Mutual Exclusion of mutex genoemd).

In [1]:
import threading, logging, time
import concurrent.futures

class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.info("Thread %s: waiting for a lock", name)
        
        # lock for the thread
        self._lock.acquire()
        try:
            logging.info("Thread %s: acquired a lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
        finally:
            logging.info("Thread %s: released a lock", name)
            self._lock.release()
        logging.info("Thread %s: finishing update", name)

In [2]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

19:15:06: Testing update. Starting value is 0.
19:15:06: Thread 0: starting update
19:15:06: Thread 1: starting update
19:15:06: Thread 0: waiting for a lock
19:15:06: Thread 1: waiting for a lock
19:15:06: Thread 0: acquired a lock
19:15:06: Thread 0: released a lock
19:15:06: Thread 0: finishing update
19:15:06: Thread 1: acquired a lock
19:15:07: Thread 1: released a lock
19:15:07: Thread 1: finishing update
19:15:07: Testing update. Ending value is 2.


Doordat er gebruik gemaakt is van $\textbf{acquire()}$ en $\textbf{release()}$ krijgt constant maar één Thread toegang tot de value, waarmee ook uiteindelijk de Race condition mee verholpen wordt.